# SCRAPING OF DATA

In [1]:
import pandas as pd
import re 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time 
from lxml import html
import requests
from email import header
import numpy as np
from bs4 import BeautifulSoup

# Setting headers as fake agents
header= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.76'}



#Automatic opening of Chrome Web_browser
driver = webdriver.Chrome()

#Opening a specific page using Link
driver.get("https://www.zomato.com/bangalore")
time.sleep(5)
scroll_pause_time = 3
screen_height = driver.execute_script("return window.screen.height;")

# Scorlling web_Page 
for i in range(0,20):
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
        time.sleep(scroll_pause_time)
        scroll_height = driver.execute_script('return document.body.scrollHeight')
        screen_height = scroll_height
       
# use BeautifulSoup for pulling data out of html files
soup = BeautifulSoup(driver.page_source,'html.parser')
di = {"id" : [],'hotel':[],"link" :[],'price_for_one':[],'rating':[],'cuisine':[]}
main_link = "https://www.zomato.com"
count = 1

# Exctracting the specific data from a class in which our required data is present
for i in soup.find_all('div',attrs={'class':'jumbo-tracker'}):
    # Continuing the loop if data is not present
    if i.next_element.next_element.next_sibling.div is None:
        continue
    else:
    # Storing the data into dictionary    
        di["id"].append(count)
        count += 1
        di['hotel'].append(i.next_element.next_element.next_sibling.div.h4.text)
        di["link"].append(main_link + i.next_element.next_element.next_sibling.get("href"))
        di['rating'].append(i.next_element.next_element.next_sibling.div.next_element.next_element.next_element.next_element.next_element.next_element.next_element.div.text)
        di['cuisine'].append(i.next_element.next_element.next_sibling.div.next_sibling.p.text)
        di['price_for_one'].append(i.next_element.next_element.next_sibling.div.next_sibling.p.next_sibling.text)

# Converting required data into Dataframe
Table1DF = pd.DataFrame(di)

# Closed the Web_Browser
driver.close()

# Taking links as a list for further use 
links = Table1DF["link"].to_list()

# Saving dataframe as a CSV file.
Table1DF.to_csv("Table_1_DF.csv")

a = links

# Defining Main List in which we append list of each required data from each webpage
restaurant = []
location = []
dish_category = []
dish_name = []
Price = []
Review = []
LON = []
LAT = []

# Defined the main dictionary which help to convert this into DataFRame
dic = {"restaurant_name" : restaurant
      ,"location" :location
      ,'dish_category':dish_category
      ,"Longitude" : LON
      ,"Latitude" : LAT
      ,'dish_name':dish_name
      ,'Price':Price
      ,"Review" : Review
     }


# Set the iterator link to automate crawling over Links List   
for i in a :
    r = [] 
    d = []
    l = []
    dc = []
    dn = []
    p = []
    re = []
    lon = []
    lat = []
    
    
    #To find Direction we using absoulte referencing method
    req= requests.get(i, headers=header)
    tree =html.fromstring(req.content)
    temp= tree.xpath('//a/@href')
    ran="https://www.google.com/maps/dir/"
    log_lat=""
    le= len(ran)
    for ll in temp:
        temp1= ll[:le]
        if ran in temp1:
            log_lat= ll
    d.append(log_lat) 
    
    ls1=d[0][52:].split(",")
    
    lat.append(ls1[0])
    lon.append(ls1[1])
    

    
    driver = webdriver.Chrome()
    url  = i
    driver.get(url)
    time.sleep(2)

   
    #To find Restraurant_name we use Xpath_Method
    RN = driver.find_elements(by=By.XPATH,value="/html/body/div[1]/div/main/div/section[3]/section/section/div/div/div/h1")
    for i in RN :
        r.append(i.text)

         

    #To find Location we use Xpath_Method
    loc = driver.find_elements(by=By.XPATH,value="/html/body/div[1]/div/main/div/section[3]/section/section/div/div/section[1]/a")
    for i in loc :
        l.append(i.text)

    #To find Dish_Category we use Xpath_Method
    dis = driver.find_elements(by=By.XPATH,value="/html/body/div[1]/div/main/div/section[4]/section/section[1]")
    for i in dis :
        dc.append(i.text) 
        
    sa = dc[0].split("\n")
    sa = sa[1:]
    dc.clear() 
    dc = sa
    
        

    #To find Dish_Name we use TAG_Name Method
    DISn = driver.find_elements(by=By.TAG_NAME,value="h4")
    for i in DISn :
        dn.append(i.text) 
       
    
    #To find Price we use Xpath_Method
    Pri = driver.find_elements(by=By.XPATH,value="//span[@class='sc-17hyc2s-1 cCiQWA']")
    for i in Pri :
        p.append(i.text)


    #To find Reviews  we use Xpath_Method
    Rev = driver.find_elements(by=By.XPATH,value="/html/body/div[1]/div/main/div/section[3]/section/section/div/div/div/section/div[3]/div[2]/div[1]")
    for i in Rev :
        re.append(i.text) 
    driver.close()
    
    #Appending the data we extracted into main list 
    restaurant.append(r[0])
    location.append(l)
    dish_category.append(dc)
    dish_name.append(dn[15:32])
    Price.append(p[15:32])
    Review.append(re)    
    LON.append(lon)
    LAT.append(lat)
    
    
# Saving results to a dataframe  
Table_2_DF = pd.DataFrame(dic)

# Saving as csv
Table_2_DF.to_csv("Table_2_DF.csv")

     
    


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\kamle\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\kamle\AppData\Local\Temp\ipykernel_8952\1008827768.py", line 31, in <module>
    scroll_height = driver.execute_script('return document.body.scrollHeight')
  File "C:\Users\kamle\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 506, in execute_script
    return self.execute(command, {"script": script, "args": converted_args})["value"]
  File "C:\Users\kamle\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 444, in execute
    self.error_handler.check_response(response)
  File "C:\Users\kamle\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 249, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchWindowException: Message: no such window: target wi

TypeError: object of type 'NoneType' has no len()

# Cleaning Part 

In [1]:
#Opening of excel file
df1 = pd.read_excel("1.xlsx")
l = []
l = df1["Cusine_Category"].to_list()
l = l[:40]
l1 = []
z=[]
for i in range (549):
    l1.append([df1["Cusines"][i]])

#To find distinct cuisine
L2 = set(L1)

    
# Count of Restraurant according to Cuisine
di = {}
for i in df1['Cusines']:
    word = i.split(',')
    for j in word:
        if j.strip() in di or j == ' ':
            continue
        else:
            di[j.strip()] = 0
for key in di:
    for j in df1['Cusines']:
        if key in j:
            di[key] += 1
            
# Converting into DataFrame             
zx = pd.DataFrame(pd.Series(di)).reset_index()

# Saving into Excel_Sheet
zx.to_excel("zx.xlsx")

NameError: name 'pd' is not defined